# Projeto WebScraping

In [1]:
# para trabalhar com diretórios / sistema operacional
import os
import re
# para nos comunicarmos com a Web
import requests

# para extrair informações de páginas HTML
import bs4
from bs4 import BeautifulSoup

# utilizada para nos indicar o caminho do executável do Python
import sys

# Para criar um Data Frame
import pandas as pd

# Controlar espera entre requisições
import time

# Gerar valores aleatórios
import random

# Produto cartesiano
from sklearn.utils.extmath import cartesian

# Renderizar HTML
import IPython

In [85]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

lista_tag_noticia = []
lista_estatisticas = []
n = 500000
p = 500
for i in range(0, n, p):
    # extraindo paginas com requests
    url = f'https://stackoverflow.com/questions?tab=active&page={i}'
    resposta = requests.get(url=url, headers=headers)
    resposta.encoding = 'utf-8'
    
    #pegando informações com soup
    soup = BeautifulSoup(resposta.text, 'html.parser')
    lista_tag_noticia.append(soup.find_all('div', class_ = 's-post-summary--content'))
    lista_estatisticas.append(soup.find_all('div', class_ = 's-post-summary--stats js-post-summary-stats'))

In [87]:
lista_votes = []

for i in range(0, n, p):
    lista_estatisticas_bkp = lista_estatisticas[j]
    for i in range(0, len(lista_estatisticas_bkp)):

        tags_questions = lista_estatisticas_bkp[i]

        vote = tags_questions.find_all('span', class_='s-post-summary--stats-item-number')
        lista_votes.append(vote)

In [86]:
lista_titulo = []
lista_tags = []
lista_data = []

for i in range(0, n, p):
    lista_tag_noticia_bkp = lista_tag_noticia[j]
    for i in range(0, len(lista_tag_noticia_bkp)):

        tags_questions = lista_tag_noticia_bkp[i]

        titulo = tags_questions.find('h3').text
        titulo = titulo.replace('\n', '') #limpa os ENTERS a esquerda e direita
        lista_titulo.append(titulo)

        tags = tags_questions.find_all('li', class_='d-inline mr4 js-post-tag-list-item')
        lista_tags.append(tags)

        data_hora = re.search(r'\d{4}-\d{2}-\d{2}', tags_questions.find('time').find('span')['title']).group()  
        lista_data.append(data_hora)

In [88]:
df = pd.DataFrame({'titulo': lista_titulo,
                   'Tags': lista_tags,
                   'data': lista_data,
                   'Estatisticas': lista_votes
                  })

In [89]:
df

,titulo,Tags,data,Estatisticas
0,background-size before IE9 [duplicate],"[[[css]], [[internet-explorer]], [[cross-brows...",2022-09-14,"[[-1], [1], [45]]"
1,Python - a function to generate numbers with t...,"[[[python]], [[distribution]]]",2022-09-14,"[[0], [1], [87]]"
2,Read Google Sheet Exported JSON by column name...,"[[[javascript]], [[google-apps-script]], [[goo...",2022-09-14,"[[0], [1], [253]]"
3,redirecting to a different page in blazor,[[[blazor]]],2022-09-14,"[[3], [1], [1k]]"
4,Dispatching Action to Update Redux State,"[[[reactjs]], [[redux]]]",2022-09-14,"[[-1], [1], [20]]"
...,...,...,...,...
14995,Media queries with edge css hack,"[[[css]], [[media-queries]]]",2022-09-14,"[[0], [2], [3k]]"
14996,Force mixed line endings in ViM,"[[[vim]], [[newline]], [[line-endings]]]",2022-09-14,"[[0], [1], [35]]"
14997,Docker CMD not working depending on the device...,"[[[docker]], [[docker-compose]], [[dockerfile]]]",2022-09-14,"[[0], [0], [47]]"
14998,How to access API in R with API documentation ...,"[[[php]], [[r]], [[api]]]",2022-09-14,"[[0], [0], [26]]"


In [90]:
# Transforma tags html em tags str
df.Tags[0]

[<li class="d-inline mr4 js-post-tag-list-item"><a aria-label="show questions tagged 'css'" aria-labelledby="tag-css-tooltip-container" class="post-tag flex--item mt0 js-tagname-css" href="/questions/tagged/css" rel="tag" title="show questions tagged 'css'">css</a></li>,
 <li class="d-inline mr4 js-post-tag-list-item"><a aria-label="show questions tagged 'internet-explorer'" aria-labelledby="tag-internet-explorer-tooltip-container" class="post-tag flex--item mt0 js-tagname-internet-explorer" href="/questions/tagged/internet-explorer" rel="tag" title="show questions tagged 'internet-explorer'">internet-explorer</a></li>,
 <li class="d-inline mr4 js-post-tag-list-item"><a aria-label="show questions tagged 'cross-browser'" aria-labelledby="tag-cross-browser-tooltip-container" class="post-tag flex--item mt0 js-tagname-cross-browser" href="/questions/tagged/cross-browser" rel="tag" title="show questions tagged 'cross-browser'">cross-browser</a></li>]

In [91]:
# Função lambda para extrair os valores da lista
extract_values = lambda x: [item.text for item in x]

# Aplicando a função lambda na coluna 'coluna1' e criando uma nova coluna 'coluna2'
df['tags'] = df['Tags'].apply(lambda x: extract_values(x))

# Exibindo o DataFrame resultante
df

,titulo,Tags,data,Estatisticas,tags
0,background-size before IE9 [duplicate],"[[[css]], [[internet-explorer]], [[cross-brows...",2022-09-14,"[[-1], [1], [45]]","[css, internet-explorer, cross-browser]"
1,Python - a function to generate numbers with t...,"[[[python]], [[distribution]]]",2022-09-14,"[[0], [1], [87]]","[python, distribution]"
2,Read Google Sheet Exported JSON by column name...,"[[[javascript]], [[google-apps-script]], [[goo...",2022-09-14,"[[0], [1], [253]]","[javascript, google-apps-script, google-sheets..."
3,redirecting to a different page in blazor,[[[blazor]]],2022-09-14,"[[3], [1], [1k]]",[blazor]
4,Dispatching Action to Update Redux State,"[[[reactjs]], [[redux]]]",2022-09-14,"[[-1], [1], [20]]","[reactjs, redux]"
...,...,...,...,...,...
14995,Media queries with edge css hack,"[[[css]], [[media-queries]]]",2022-09-14,"[[0], [2], [3k]]","[css, media-queries]"
14996,Force mixed line endings in ViM,"[[[vim]], [[newline]], [[line-endings]]]",2022-09-14,"[[0], [1], [35]]","[vim, newline, line-endings]"
14997,Docker CMD not working depending on the device...,"[[[docker]], [[docker-compose]], [[dockerfile]]]",2022-09-14,"[[0], [0], [47]]","[docker, docker-compose, dockerfile]"
14998,How to access API in R with API documentation ...,"[[[php]], [[r]], [[api]]]",2022-09-14,"[[0], [0], [26]]","[php, r, api]"


In [92]:
# Função lambda para extrair os valores da lista
extract_values = lambda x: [item.text for item in x]

# Aplicando a função lambda na coluna 'coluna1' e criando uma nova coluna 'coluna2'
df['estatisticas'] = df['Estatisticas'].apply(lambda x: extract_values(x))

# Exibindo o DataFrame resultante
df

,titulo,Tags,data,Estatisticas,tags,estatisticas
0,background-size before IE9 [duplicate],"[[[css]], [[internet-explorer]], [[cross-brows...",2022-09-14,"[[-1], [1], [45]]","[css, internet-explorer, cross-browser]","[-1, 1, 45]"
1,Python - a function to generate numbers with t...,"[[[python]], [[distribution]]]",2022-09-14,"[[0], [1], [87]]","[python, distribution]","[0, 1, 87]"
2,Read Google Sheet Exported JSON by column name...,"[[[javascript]], [[google-apps-script]], [[goo...",2022-09-14,"[[0], [1], [253]]","[javascript, google-apps-script, google-sheets...","[0, 1, 253]"
3,redirecting to a different page in blazor,[[[blazor]]],2022-09-14,"[[3], [1], [1k]]",[blazor],"[3, 1, 1k]"
4,Dispatching Action to Update Redux State,"[[[reactjs]], [[redux]]]",2022-09-14,"[[-1], [1], [20]]","[reactjs, redux]","[-1, 1, 20]"
...,...,...,...,...,...,...
14995,Media queries with edge css hack,"[[[css]], [[media-queries]]]",2022-09-14,"[[0], [2], [3k]]","[css, media-queries]","[0, 2, 3k]"
14996,Force mixed line endings in ViM,"[[[vim]], [[newline]], [[line-endings]]]",2022-09-14,"[[0], [1], [35]]","[vim, newline, line-endings]","[0, 1, 35]"
14997,Docker CMD not working depending on the device...,"[[[docker]], [[docker-compose]], [[dockerfile]]]",2022-09-14,"[[0], [0], [47]]","[docker, docker-compose, dockerfile]","[0, 0, 47]"
14998,How to access API in R with API documentation ...,"[[[php]], [[r]], [[api]]]",2022-09-14,"[[0], [0], [26]]","[php, r, api]","[0, 0, 26]"


In [93]:
df = df.drop(columns = ['Tags', 'Estatisticas'])

In [94]:
df

,titulo,data,tags,estatisticas
0,background-size before IE9 [duplicate],2022-09-14,"[css, internet-explorer, cross-browser]","[-1, 1, 45]"
1,Python - a function to generate numbers with t...,2022-09-14,"[python, distribution]","[0, 1, 87]"
2,Read Google Sheet Exported JSON by column name...,2022-09-14,"[javascript, google-apps-script, google-sheets...","[0, 1, 253]"
3,redirecting to a different page in blazor,2022-09-14,[blazor],"[3, 1, 1k]"
4,Dispatching Action to Update Redux State,2022-09-14,"[reactjs, redux]","[-1, 1, 20]"
...,...,...,...,...
14995,Media queries with edge css hack,2022-09-14,"[css, media-queries]","[0, 2, 3k]"
14996,Force mixed line endings in ViM,2022-09-14,"[vim, newline, line-endings]","[0, 1, 35]"
14997,Docker CMD not working depending on the device...,2022-09-14,"[docker, docker-compose, dockerfile]","[0, 0, 47]"
14998,How to access API in R with API documentation ...,2022-09-14,"[php, r, api]","[0, 0, 26]"


In [95]:
df[['votes', 'answers', 'views']] = df['estatisticas'].apply(lambda x: pd.Series(x))

In [96]:
df

,titulo,data,tags,estatisticas,votes,answers,views
0,background-size before IE9 [duplicate],2022-09-14,"[css, internet-explorer, cross-browser]","[-1, 1, 45]",-1,1,45
1,Python - a function to generate numbers with t...,2022-09-14,"[python, distribution]","[0, 1, 87]",0,1,87
2,Read Google Sheet Exported JSON by column name...,2022-09-14,"[javascript, google-apps-script, google-sheets...","[0, 1, 253]",0,1,253
3,redirecting to a different page in blazor,2022-09-14,[blazor],"[3, 1, 1k]",3,1,1k
4,Dispatching Action to Update Redux State,2022-09-14,"[reactjs, redux]","[-1, 1, 20]",-1,1,20
...,...,...,...,...,...,...,...
14995,Media queries with edge css hack,2022-09-14,"[css, media-queries]","[0, 2, 3k]",0,2,3k
14996,Force mixed line endings in ViM,2022-09-14,"[vim, newline, line-endings]","[0, 1, 35]",0,1,35
14997,Docker CMD not working depending on the device...,2022-09-14,"[docker, docker-compose, dockerfile]","[0, 0, 47]",0,0,47
14998,How to access API in R with API documentation ...,2022-09-14,"[php, r, api]","[0, 0, 26]",0,0,26


In [97]:
df = df.drop(columns = ['estatisticas'])

In [98]:
df

,titulo,data,tags,votes,answers,views
0,background-size before IE9 [duplicate],2022-09-14,"[css, internet-explorer, cross-browser]",-1,1,45
1,Python - a function to generate numbers with t...,2022-09-14,"[python, distribution]",0,1,87
2,Read Google Sheet Exported JSON by column name...,2022-09-14,"[javascript, google-apps-script, google-sheets...",0,1,253
3,redirecting to a different page in blazor,2022-09-14,[blazor],3,1,1k
4,Dispatching Action to Update Redux State,2022-09-14,"[reactjs, redux]",-1,1,20
...,...,...,...,...,...,...
14995,Media queries with edge css hack,2022-09-14,"[css, media-queries]",0,2,3k
14996,Force mixed line endings in ViM,2022-09-14,"[vim, newline, line-endings]",0,1,35
14997,Docker CMD not working depending on the device...,2022-09-14,"[docker, docker-compose, dockerfile]",0,0,47
14998,How to access API in R with API documentation ...,2022-09-14,"[php, r, api]",0,0,26
